# aT 농산품 예측 base line

&nbsp;

## 모듈 가져오기

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
from numba import jit,njit

test_num = 8

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xd

ImportError: initialization failed

## 입력 shape 및 형태 정의 함수

In [ ]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [ ]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [ ]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [ ]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=50)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [ ]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.002, verbose = 1):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[256],
    mlp_dropout=0.3,
    dropout=0.3,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [ ]:
@jit
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [ ]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1000
batch = 64
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

&nbsp;

## Train 과정

In [ ]:
for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)

    # nan 처리
    df = df.fillna(0)

    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch)
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

100%|██████████| 1419/1419 [00:00<00:00, 80219.39it/s]


Epoch 1/1000


2022-09-03 13:34:08.969114: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/17 [==============================] - ETA: 0s - loss: 6615078912.0000

2022-09-03 13:34:12.683449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-03 16:14:09.623527: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/17 [==============================] - ETA: 0s - loss: 38351646720.0000

2022-09-03 16:14:10.796962: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 796385856.00000, saving model to ./check/transformer-9-1000-64.h5
17/17 [==============================] - 2s 38ms/step - loss: 38351646720.0000 - val_loss: 796385856.0000
Epoch 2/1000
15/17 [========================>.....] - ETA: 0s - loss: 8456862208.0000 
Epoch 2: val_loss improved from 796385856.00000 to 322290432.00000, saving model to ./check/transformer-9-1000-64.h5
17/17 [==============================] - 0s 10ms/step - loss: 7504525312.0000 - val_loss: 322290432.0000
Epoch 3/1000
15/17 [========================>.....] - ETA: 0s - loss: 2361852672.0000
Epoch 3: val_loss improved from 322290432.00000 to 87895928.00000, saving model to ./check/transformer-9-1000-64.h5
17/17 [==============================] - 0s 6ms/step - loss: 2102343424.0000 - val_loss: 87895928.0000
Epoch 4/1000
14/17 [======================>.......] - ETA: 0s - loss: 807033664.0000 
Epoch 4: val_loss improved from 87895928.00000 to 40364076.00000, saving model to ./chec

100%|██████████| 37/37 [2:46:28<00:00, 269.95s/it]


&nbsp;

## Test 과정

In [ ]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)

        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]

        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))
        
        # model test
        if os.path.exists(f'./model_output/{test_num}') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]2022-09-03 16:20:34.220451: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 65ms/step


2022-09-03 16:20:34.852244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 16:20:35.012892: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 41ms/step


2022-09-03 16:20:35.295450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 16:20:35.409434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-03 16:23:19.058198: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 16:23:19.247383: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 47ms/step


2022-09-03 16:23:19.372978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 16:23:19.490157: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 50ms/step


2022-09-03 16:23:19.709376: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 10/10 [02:46<00:00, 16.64s/it]


&nbsp;

## 정답 제출 파일생성

In [ ]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  7.67it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [ ]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.1250,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.2800,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,3429.2595,1722.205100,4913.592300,3008.009800,203.699890,5062.928700,2341.3052,3796.602000,3083.117700,984.339050,...,5929.406000,197.053560,498.627350,3.637097,3623.807000,3544.880100,3423.020000,2066.847700,1749.395600,1341.384500
2,3651.4172,858.423100,3939.052200,1863.934900,289.197400,4127.362300,2344.7898,3316.781700,2400.129400,981.649230,...,5093.233400,207.962680,485.291470,6.192522,2662.755600,3144.542700,3209.933000,1982.848900,2001.878000,1351.499600
3,1591.9890,850.820100,416.000030,874.806300,249.542760,5953.684000,2263.2454,1807.513700,548.963100,918.387300,...,2468.795400,168.045180,484.868000,7.044287,871.566470,3035.008300,261.303400,1895.705100,590.573800,1334.249000
4,3868.9810,1785.546500,4699.410000,2792.625500,206.342710,4765.273000,2298.8115,4117.442400,3254.983000,941.857600,...,7472.506000,263.159500,487.952180,4.285394,3230.306600,3218.833300,3235.516800,2137.987800,2127.481200,1322.061000
5,3629.1191,1753.659700,5137.398400,2523.499000,258.006800,3735.279300,2309.1907,3530.287800,3394.067900,972.181800,...,7073.942400,274.256160,491.616600,3.711823,3025.676300,3665.448200,3759.558300,1998.610600,2137.984400,1311.987400
6,3458.6763,1755.409700,5052.750000,3072.253400,255.432330,3888.617200,2220.8965,4482.356400,3393.155800,1018.828600,...,7375.593800,281.884160,492.400050,3.122609,3029.837000,3286.544700,3446.579300,2017.878500,1991.814200,1351.516200
7,3619.6628,1769.038000,5035.697300,3345.758800,248.304630,4333.121000,2218.8713,3831.141000,3481.554400,1023.270300,...,6816.775400,186.781360,496.375180,3.809639,3590.745000,3353.723600,3659.342500,1800.616100,2051.114700,1343.413100
8,3237.4248,1634.961800,4909.213000,3070.391800,122.713700,4871.543500,2237.8890,3153.725600,2677.266400,978.169250,...,5405.922000,205.897340,490.155180,6.595932,3670.042500,3366.239300,3333.375500,1976.053000,1721.468400,1337.371800
9,3760.1038,840.950500,3319.093800,1565.094700,248.704540,4593.784000,2250.8726,2731.081300,2597.363800,1157.656400,...,4292.319000,203.719070,493.290620,1.834429,2650.499000,3304.683800,3331.544200,2100.807400,1943.630000,1359.145000


- 변동률 계산 

In [ ]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [ ]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [ ]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,-0.114144,0.264359,0.688645,-0.115311,-0.948402,-0.452890,-0.138364,0.129594,-0.372317,-0.160850,...,-0.548922,-0.997678,-0.002658,-0.350125,-0.390834,-0.604856,-0.397819,-0.357474,0,d+1
1,-0.056756,-0.369788,0.353727,-0.451796,-0.926745,-0.553989,-0.137082,-0.013165,-0.511364,-0.163143,...,-0.560986,-0.996046,-0.267158,-0.423518,-0.428755,-0.620915,-0.310909,-0.352629,0,d+2
2,-0.588753,-0.375370,-0.857034,-0.742710,-0.936790,-0.356633,-0.167092,-0.462214,-0.888238,-0.217074,...,-0.561369,-0.995503,-0.760128,-0.443599,-0.953498,-0.637575,-0.796711,-0.360892,0,d+3
3,-0.000554,0.310861,0.615038,-0.178658,-0.947732,-0.485055,-0.154003,0.225053,-0.337327,-0.197065,...,-0.558579,-0.997264,-0.110957,-0.409898,-0.424202,-0.591255,-0.267674,-0.366730,0,d+4
4,-0.062516,0.287451,0.765560,-0.257811,-0.934646,-0.596358,-0.150183,0.050359,-0.309011,-0.171214,...,-0.555264,-0.997630,-0.167275,-0.328021,-0.330943,-0.617902,-0.264058,-0.371556,0,d+5
5,-0.106545,0.288736,0.736469,-0.096416,-0.935298,-0.579788,-0.182677,0.333625,-0.309197,-0.131447,...,-0.554555,-0.998006,-0.166130,-0.397485,-0.386641,-0.614218,-0.314373,-0.352621,0,d+6
6,-0.064958,0.298741,0.730609,-0.015975,-0.937103,-0.531754,-0.183422,0.139871,-0.291200,-0.127661,...,-0.550959,-0.997568,-0.011757,-0.385169,-0.348777,-0.655755,-0.293961,-0.356503,0,d+7
7,-0.163699,0.200309,0.687140,-0.096964,-0.968916,-0.473571,-0.176423,-0.061679,-0.454943,-0.166109,...,-0.556586,-0.995789,0.010067,-0.382875,-0.406787,-0.622214,-0.407432,-0.359397,0,d+8
8,-0.028679,-0.382615,0.140667,-0.539688,-0.937002,-0.503587,-0.171645,-0.187427,-0.471210,-0.013096,...,-0.553750,-0.998829,-0.270531,-0.394160,-0.407113,-0.598364,-0.330960,-0.348967,0,d+9
9,-0.672836,-0.326801,-0.860667,-0.622100,-0.934916,-0.660181,-0.186976,-0.566823,-0.930042,-0.204605,...,-0.556212,-0.997540,-0.669348,-0.418158,-0.950180,-0.659040,-0.748911,-0.351788,0,d+10


- 정답 양식으로 변경

In [ ]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [ ]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.114144,0.264359,0.688645,-0.115311,-0.948402,-0.452890,-0.138364,0.129594,...,-0.313791,-0.672672,-0.548922,-0.997678,-0.002658,-0.350125,-0.390834,-0.604856,-0.397819,-0.357474
1,0,d+2,-0.056756,-0.369788,0.353727,-0.451796,-0.926745,-0.553989,-0.137082,-0.013165,...,-0.410561,-0.654550,-0.560986,-0.996046,-0.267158,-0.423518,-0.428755,-0.620915,-0.310909,-0.352629
2,0,d+3,-0.588753,-0.375370,-0.857034,-0.742710,-0.936790,-0.356633,-0.167092,-0.462214,...,-0.714287,-0.720858,-0.561369,-0.995503,-0.760128,-0.443599,-0.953498,-0.637575,-0.796711,-0.360892
3,0,d+4,-0.000554,0.310861,0.615038,-0.178658,-0.947732,-0.485055,-0.154003,0.225053,...,-0.135208,-0.562862,-0.558579,-0.997264,-0.110957,-0.409898,-0.424202,-0.591255,-0.267674,-0.366730
4,0,d+5,-0.062516,0.287451,0.765560,-0.257811,-0.934646,-0.596358,-0.150183,0.050359,...,-0.181334,-0.544429,-0.555264,-0.997630,-0.167275,-0.328021,-0.330943,-0.617902,-0.264058,-0.371556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.361958,-0.302284,0.519507,-0.167434,-1.010361,-0.275291,-0.949650,NaN,...,-0.051267,-0.634277,-0.559977,-1.563935,0.033133,-0.292513,-0.541629,0.169197,-0.105083,-0.475296
146,9,d+12,-0.289390,-0.290107,0.398536,-0.111935,-0.961241,-0.389582,-0.924096,NaN,...,-0.009392,-0.587255,-0.550982,0.301672,0.040146,-0.256445,-0.495237,0.284333,-0.067798,-0.467509
147,9,d+13,-0.372996,-0.283836,0.469361,-0.113214,-0.925611,-0.303679,-0.948111,NaN,...,-0.071160,-0.524506,-0.556154,-0.275004,-0.072691,-0.346255,-0.589973,-0.006313,-0.219437,-0.468223
148,9,d+14,-0.329822,-0.279388,0.477036,-0.076001,-1.023138,-0.170275,-0.948903,NaN,...,-0.020328,-0.630661,-0.551992,-0.546501,0.029348,-0.187430,-0.544834,0.166796,-0.162955,-0.467444


- 제출 양식

In [ ]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [ ]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.114144,0.264359,0.688645,-0.115311,-0.948402,-0.452890,-0.138364,0.129594,...,-0.313791,-0.672672,-0.548922,-0.997678,-0.002658,-0.350125,-0.390834,-0.604856,-0.397819,-0.357474
1,0,d+2,-0.056756,-0.369788,0.353727,-0.451796,-0.926745,-0.553989,NaN,-0.013165,...,-0.410561,-0.654550,-0.560986,-0.996046,-0.267158,-0.423518,-0.428755,-0.620915,-0.310909,-0.352629
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,-0.167092,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,-0.000554,0.310861,0.615038,-0.178658,-0.947732,-0.485055,NaN,0.225053,...,-0.135208,-0.562862,-0.558579,-0.997264,-0.110957,-0.409898,-0.424202,-0.591255,-0.267674,-0.366730
4,0,d+5,-0.062516,0.287451,0.765560,-0.257811,-0.934646,-0.596358,NaN,0.050359,...,-0.181334,-0.544429,-0.555264,-0.997630,-0.167275,-0.328021,-0.330943,-0.617902,-0.264058,-0.371556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.361958,-0.302284,0.519507,-0.167434,-1.010361,-0.275291,-0.949650,NaN,...,-0.051267,-0.634277,-0.559977,-1.563935,0.033133,-0.292513,-0.541629,0.169197,-0.105083,-0.475296
146,9,d+12,-0.289390,-0.290107,0.398536,-0.111935,-0.961241,-0.389582,-0.924096,NaN,...,-0.009392,-0.587255,-0.550982,0.301672,0.040146,-0.256445,-0.495237,0.284333,-0.067798,-0.467509
147,9,d+13,-0.372996,-0.283836,0.469361,-0.113214,-0.925611,-0.303679,-0.948111,NaN,...,-0.071160,-0.524506,-0.556154,-0.275004,-0.072691,-0.346255,-0.589973,-0.006313,-0.219437,-0.468223
148,9,d+14,-0.329822,-0.279388,0.477036,-0.076001,-1.023138,-0.170275,-0.948903,NaN,...,-0.020328,-0.630661,-0.551992,-0.546501,0.029348,-0.187430,-0.544834,0.166796,-0.162955,-0.467444
